This is a Python example demonstrating use of the USEEIO API.
It assumes that the API is running on the local machine and queries it from there.

In [1]:
#BASE URL AND KEY
#Use the requests library
import requests as r
#
import pandas as pd
#Set the base URL.
base_url = 'http://localhost:8080/api/'
#Add a key for the request hhttps://api.edap-cluster.com/useeio/api/eaders. No keys are needed by default, so it's blank
api_headers = {}

In [5]:
#AVAILABLE MODELS
#Show the available models in the native JSON output
models = r.get(base_url+'models',headers=api_headers)
models_json = models.json()
models_json

[{'id': 'ExampleModel',
  'name': 'Example Model',
  'location': 'US',
  'description': 'Example IO model from the IO-Model-Builder'}]

#Use the first available model
model_name = models_json[0]['id']
print('The first model name is ' + str(model_name) + '.')

In [7]:
#MODEL NAME
#Use the first available model
model_name = models_json[0]['id']
print('The first model name is ' + str(model_name) + '.')

The first model name is ExampleModel.


In [8]:
#MODEL SECTORS
#Get the sectors
model_sectors = r.get(base_url+'/'+model_name+'/sectors',headers=api_headers)
model_sectors_json = model_sectors.json()
model_sector_ids = []
for i in model_sectors_json:
    model_sector_ids.append(i['id'])
first_sector_id = model_sector_ids[0]
first_sector_name = model_sectors_json[0]['name']
print('The '+ model_name +' model has ' + str(len(model_sectors_json)) + ' sectors.')
print('The first sector name is \"' + first_sector_name + '\" with an ID of \"' + first_sector_id + '\"')

The ExampleModel model has 6 sectors.
The first sector name is "Agriculture & Forestry" with an ID of "1/agriculture & forestry/us"


In [5]:
#DIRECT REQUIREMENT MATRIX
#Get the first column of the direct requirement matrix
A0_response = r.get(base_url+model_name+'/matrix/A?col=0',headers=api_headers)
A0 = A0_response.json()
#Convert it to a dataframe
A0_df = pd.DataFrame(A0,columns=[first_sector_id],index=model_sector_ids)
#Sort it by descending order of requirements
A0_df = A0_df.sort_values(by=first_sector_id,ascending=False)
print('Table. Top 10 direct inputs into ' + first_sector_name +'.')
A0_df.head(10)

Table. Top 10 direct inputs into Agriculture & Forestry.


,1/agriculture & forestry/us
3/construction & manufacturing/us,0.210018
1/agriculture & forestry/us,0.204289
4/trade & transport/us,0.092089
5/information & finance & services/us,0.066902
2/mining & utilities/us,0.021635
6/government & other/us,0.001980


In [6]:
#TOTAL REQUIREMENT MATRIX
#Get the first column of the total requirement matrix
L0_response = r.get(base_url+model_name+'/matrix/L?col=0',headers=api_headers)
L0 = L0_response.json()
#Convert it to a dataframe
L0_df = pd.DataFrame(L0,columns=[first_sector_id],index=model_sector_ids)
#Sort it by descending order of requirements
L0_df = L0_df.sort_values(by=first_sector_id,ascending=False)
print('Table. Top 10 total requirements into ' + first_sector_name +'.')
L0_df.head(10)

Table. Top 10 total requirements into Agriculture & Forestry.


,1/agriculture & forestry/us
1/agriculture & forestry/us,1.278440
3/construction & manufacturing/us,0.451877
5/information & finance & services/us,0.238329
4/trade & transport/us,0.178166
2/mining & utilities/us,0.086659
6/government & other/us,0.006564


In [7]:
#ELEMENTARY FLOWS
model_flows = r.get(base_url+model_name+'/flows',headers=api_headers)
model_flows_json = model_flows.json()
first_flow_id = model_flows_json[0]['id']
first_flow_name = model_flows_json[0]['name']
model_flow_ids = []
for i in model_flows_json:
    model_flow_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_flows_json)) + ' elementary flows.')
print('The first flow name is \"' + first_flow_name + '\" with an ID of \"' + first_flow_id + '\".')

The ExampleModel model has 15 elementary flows.
The first flow name is "Carbon dioxide" with an ID of "air/unspecified/carbon dioxide/kg".


In [8]:
#RESOURCE USE
#Query the first row satellite table matrix, B. This provides the resource use or emission intensities for all sectors for
#the first elementary flow in the model
B0_response = r.get(base_url+model_name+'/matrix/B?row=0',headers=api_headers)
B0 = B0_response.json()
#Query the data quality score for the accompanying satellite table values from B_dqi
Bdqi0_response = r.get(base_url+model_name+'/matrix/B_dqi?row=0',headers=api_headers)
try:
    Bdqi0 = Bdqi0_response.json()
except:
    Bdqi0 = [0]*len(B0)
#Assign a name to the intensities
Bvaluename = 'unit/$ gross output'
#Assign a more readable name for data quality scores
dqname = '(Reliability,Time Corr,Geo Corr,Tech Corr,Data Collection)'

#Convert it to a dataframe
B0_df = pd.DataFrame({Bvaluename:B0,dqname:Bdqi0},index=model_sector_ids)
#Sort it by descending order of intensities
B0_df = B0_df.sort_values(by=Bvaluename,ascending=False)
print('Table. Top 5 direct intensities (unit/$) of ' + first_flow_name +' with DQ scores.')
B0_df.head(5)

Table. Top 5 direct intensities (unit/$) of Carbon dioxide with DQ scores.


,unit/$ gross output,"(Reliability,Time Corr,Geo Corr,Tech Corr,Data Collection)"
3/construction & manufacturing/us,29.836795,"(3,1,4,3,1)"
6/government & other/us,11.213541,"(3,4,4,2,5)"
2/mining & utilities/us,9.297182,"(5,3,1,5,2)"
4/trade & transport/us,5.892112,"(2,3,5,5,4)"
5/information & finance & services/us,1.691867,"(2,4,3,2,3)"


In [9]:
#INDICATORS
model_indictrs_response = r.get(base_url+model_name+'/indicators',headers=api_headers)
model_indictrs_json = model_indictrs_response.json()
first_indictr_id = model_indictrs_json[0]['id']
model_indictrs_names = []
model_indictrs_ids = []
for i in model_indictrs_json:
    model_indictrs_names.append(i['name'])
    model_indictrs_ids.append(i['id'])
print('The '+ model_name +' model has ' + str(len(model_indictrs_json)) + ' indicators. The names are:')
print(model_indictrs_names)

The ExampleModel model has 4 indicators. The names are:
['Smog Formation- SFP', 'Greenhouse Gases', 'Human Health - non-carcenogenic', 'Human Health Tox-Carcinogenic']


In [10]:
#CHARACTERIZATION FACTORS
#Query the first row of the characterization factor(CF) matrix, B. This provides the CFs for the first indicator
C0_response = r.get(base_url+model_name+'/matrix/C?row=0',headers=api_headers)
C0 = C0_response.json()
#Convert it to a dataframe
C0_df = pd.DataFrame(C0,columns=[first_indictr_id],index=model_flow_ids)
#Sort it by descending order
C0_df = C0_df.sort_values(by=first_indictr_id,ascending=False)
print('Table. Top 5 characterization factors for ' + model_indictrs_names[0] +'.')
C0_df.head(5)

Table. Top 5 characterization factors for Smog Formation- SFP.


,SMOG
air/unspecified/methane/kg,0.039169
"air/unspecified/ethane, 1,1,1,2-tetrafluoro-, hfc-134a/kg",0.018716
air/unspecified/carbon dioxide/kg,0.000000
air/unspecified/dinitrogen monoxide/kg,0.000000
"air/unspecified/ethane, pentafluoro-, hfc-125/kg",0.000000


In [11]:
#DIRECT IMPACTS
#Get top direct impacts for first indicator
#Query the first row of the direct impacts matrix, D. This provides the direct impacts for the first indicator.
D0_response = r.get(base_url+model_name+'/matrix/D?row=0',headers=api_headers)
D0 = D0_response.json()
#Get the accompanying data quality scores
D0dqi_response = r.get(base_url+model_name+'/matrix/D_dqi?row=0',headers=api_headers)
try:
    D0dqi = D0dqi_response.json()
except: 
    D0dqi = [0]*len(D0)
#Convert it to a dataframe
D0_df = pd.DataFrame({first_indictr_id:D0,dqname:D0dqi},index=model_sector_ids)
#Sort it by descending order
D0_df = D0_df.sort_values(by=first_indictr_id,ascending=False)
print('Table. Top 10 direct sector impacts per dollar for ' + model_indictrs_names[0] +' with DQ scores.')
D0_df.head(10)

Table. Top 10 direct sector impacts per dollar for Smog Formation- SFP with DQ scores.


,SMOG,"(Reliability,Time Corr,Geo Corr,Tech Corr,Data Collection)"
1/agriculture & forestry/us,0.006826,"(2,5,1,3,4)"
2/mining & utilities/us,0.004270,"(2,3,5,3,1)"
4/trade & transport/us,0.002867,"(1,5,3,4,1)"
5/information & finance & services/us,0.001986,"(3,3,5,4,4)"
3/construction & manufacturing/us,0.001014,"(1,3,5,2,1)"
6/government & other/us,0.000007,"(1,3,4,4,1)"


In [12]:
#LIFE CYCLE IMPACTS PER DOLLAR
#Get top life cycle impacts per $ for first indicator
U0_response = r.get(base_url+model_name+'/matrix/U?row=0',headers=api_headers)
U0 = U0_response.json()
U0dqi_response = r.get(base_url+model_name+'/matrix/U_dqi?row=0',headers=api_headers)
try:
    U0dqi = U0dqi_response.json()
except:
    U0dqi = [0]*len(U0)
#Convert it to a dataframe
U0_df = pd.DataFrame({first_indictr_id:U0,dqname:U0dqi},index=model_sector_ids)
#Sort it by descending order
U0_df = U0_df.sort_values(by=first_indictr_id,ascending=False)
print('Table. Top 10 life cycle impacts sector impacts per dollar for ' + model_indictrs_names[0] +'.')
U0_df.head(10)

Table. Top 10 life cycle impacts sector impacts per dollar for Smog Formation- SFP.


,SMOG,"(Reliability,Time Corr,Geo Corr,Tech Corr,Data Collection)"
1/agriculture & forestry/us,0.010540,"(2,5,2,3,4)"
2/mining & utilities/us,0.005967,"(2,3,5,3,1)"
4/trade & transport/us,0.004290,"(1,5,3,4,2)"
3/construction & manufacturing/us,0.003840,"(2,3,4,3,2)"
5/information & finance & services/us,0.003391,"(3,3,5,4,4)"
6/government & other/us,0.001333,"(2,3,4,3,3)"


In [13]:
#DEMAND VECTORS
#See the demand vectors available for the model in their JSON format
demands_response = r.get(base_url+model_name+'/demands',headers=api_headers)
demands = demands_response.json()
demands

[{'id': '2016_us_testdemand',
  'location': 'US',
  'system': 'Full System',
  'type': 'TestDemand',
  'year': 2016}]

In [14]:
#DEMAND VECTOR IN NATIVE FORMAT
#See the first demand vector in its native format, which has to be created if one wants to calculate the model results with a custom vector
first_demand_id = demands[0]['id']
first_demand_id
y0_response = r.get(base_url+model_name+'/demands/'+first_demand_id,headers=api_headers)
y0 = y0_response.json()
if len(y0)>5:
    print(y0[0:5])
else:
    print(y0)

[{'amount': 1.0, 'sector': '3/construction & manufacturing/us'}]


In [15]:
#SECTORS RANKED BY DEMAND
#See the top 10 sectors ranked by demand for the first demand vector
y0_df = pd.DataFrame(y0)
y0_df = y0_df.sort_values(by='amount',ascending=False)
print('Table. Top 10 sectors ranked by demand ' + first_demand_id  +'.')
y0_df.head(10)

Table. Top 10 sectors ranked by demand 2016_us_testdemand.


,amount,sector
0,1.0,3/construction & manufacturing/us


In [16]:
#SECTORS IN ROWS. 
#Use this same demand for a model calculation. Use the direct perspective calculation. 
#Show a transposed view with indicators in the columns and sectors in the rows
data_to_post = {"perspective": "direct"}
data_to_post["demand"] = y0
result1_response = r.post(base_url+model_name+'/calculate',headers=api_headers,json=data_to_post)
result1 = result1_response.json()
result1_df = pd.DataFrame(data=result1['data'],columns=result1['sectors'],index=result1['indicators']).transpose()
result1_df = result1_df.sort_values(by=first_indictr_id,ascending=False)
print('Top ten sectors by direct perspective impact for indicator '+ model_indictrs_names[0] + ' using demand vector ' +  first_demand_id + ', with 4 additional indicator results presented.')
result1_df[model_indictrs_ids[0:5]].head(10)

Top ten sectors by direct perspective impact for indicator Smog Formation- SFP using demand vector 2016_us_testdemand, with 4 additional indicator results presented.


,SMOG,GHG,HTOX-NC,HTOX-HC
3/construction & manufacturing/us,1.591011e-03,52.550475,3.133217e-12,8.600987e-12
2/mining & utilities/us,7.595640e-04,2.218259,3.893705e-16,1.068860e-15
5/information & finance & services/us,5.325586e-04,0.832723,4.795932e-14,1.316530e-13
1/agriculture & forestry/us,5.031686e-04,-2.443094,4.755755e-17,1.305501e-16
4/trade & transport/us,4.533499e-04,1.262775,4.845930e-14,1.330255e-13
6/government & other/us,5.889780e-08,0.091000,3.671321e-17,1.007813e-16


In [17]:
#SECTORS IN ROWS - FINAL PERSPECTIVE
#Now show the same results with the final perspective
data_to_post["perspective"] = "final"
result2_response = r.post(base_url+model_name+'/calculate',headers=api_headers,json=data_to_post)
result2 = result2_response.json()
result2_df = pd.DataFrame(data=result2['data'],columns=result2['sectors'],index=result2['indicators']).transpose()
result2_df = result2_df.sort_values(by=first_indictr_id,ascending=False)
print('Top ten sectors by final perspective impact for indicator '+ model_indictrs_names[0] + ' using demand vector ' +  first_demand_id + ', with 4 additional indicator results presented.')
result2_df[model_indictrs_ids[0:5]].head(10)

Top ten sectors by final perspective impact for indicator Smog Formation- SFP using demand vector 2016_us_testdemand, with 4 additional indicator results presented.


,SMOG,GHG,HTOX-NC,HTOX-HC
3/construction & manufacturing/us,0.00384,54.512138,3.230109e-12,8.866966e-12
1/agriculture & forestry/us,0.00000,0.000000,0.000000e+00,0.000000e+00
2/mining & utilities/us,0.00000,0.000000,0.000000e+00,0.000000e+00
4/trade & transport/us,0.00000,0.000000,0.000000e+00,0.000000e+00
5/information & finance & services/us,0.00000,0.000000,0.000000e+00,0.000000e+00
6/government & other/us,0.00000,0.000000,0.000000e+00,0.000000e+00


In [18]:
#DEMAND VECTOR FOR 1 MILLION
#In place of the first available demand vector, prepare a demand vector for 1 millon USD demand for the first model sector
#Show the revised json object to be posted
y1 = {}
y1["sector"] = first_sector_id
y1["amount"] = 1E6
data_to_post["demand"] = [y1]
data_to_post["perspective"] = "direct"
data_to_post

{'perspective': 'direct',
 'demand': [{'sector': '1/agriculture & forestry/us', 'amount': 1000000.0}]}

In [19]:
#DEMAND VECTOR DIRECT PERSPECTIVE
#Calculate direct perspectiv result using this new prepared demand vector
result3_response = r.post(base_url+model_name+'/calculate',headers=api_headers,json=data_to_post)
result3 = result3_response.json()
result3_df = pd.DataFrame(data=result3['data'],columns=result3['sectors'],index=result3['indicators']).transpose()
result3_df = result3_df.sort_values(by=first_indictr_id,ascending=False)
print('Top ten sectors by direct perspective impact for indicator '+ model_indictrs_names[0] + 
      ' for 1 M USD demand for ' +  first_sector_name + ', with 4 additional indicator results presented.')
result3_df[model_indictrs_ids[0:5]].head(10)

Top ten sectors by direct perspective impact for indicator Smog Formation- SFP for 1 M USD demand for Agriculture & Forestry, with 4 additional indicator results presented.


,SMOG,GHG,HTOX-NC,HTOX-HC
1/agriculture & forestry/us,8727.209162,-4.237425e+07,8.248620e-10,2.264327e-09
4/trade & transport/us,510.853977,1.422948e+06,5.460600e-08,1.498988e-07
5/information & finance & services/us,473.426222,7.402623e+05,4.263418e-08,1.170350e-07
3/construction & manufacturing/us,458.064295,1.512968e+07,9.020770e-07,2.476290e-06
2/mining & utilities/us,370.063998,1.080748e+06,1.897036e-10,5.207549e-10
6/government & other/us,0.048109,7.433061e+04,2.998807e-11,8.232018e-11
